### Load data

In [1]:
import json

with open('./mastercourse.json') as f:
    texts = json.load(f)

In [2]:
print (f'Now you have {len(texts)} documents')

Now you have 1460 documents


### Create embeddings of your documents to get ready for semantic search

In [3]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/home/aum/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [9]:
OPENAI_API_KEY = '...'
PINECONE_API_KEY = '...'
PINECONE_API_ENV = '...'

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [11]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "..."

In [12]:
docsearch = Pinecone.from_texts([t for t in texts], embeddings, index_name=index_name)

### Query those docs to get your answer back

In [13]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [23]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [32]:
query = "What does Hinduism say about suice?"
docs = docsearch.similarity_search(query, include_metadata=True)
docs

[Document(page_content='Lesson 229 from Living with Siva\n\nQuestions On Suicide \nAnother very serious issue faced today in every society is suicide. The percentages are too high to ignore the problem that exists in far too many Hindu communities. Well, we can advise, as many elders do: "Don\'t kill yourself." After all, they became elders by avoiding such extreme solutions. But do those who are all wrought up with emotion and confusion listen to such advice? No. Many die needlessly at their own hand. How selfish. How sad. But it is happening every day. Suicide does not solve problems. It only magnifies future problems in the Antarloka--the subtle, nonphysical astral world we live in before we incarnate--and in the next life. Suicide only accelerates the intensity of karma, bringing a series of immediate lesser births and requiring several lives for the soul to return to the evolutionary point that existed at the moment of suicide, at which time the still existing karmic entanglement 

In [21]:
# query = "What is the collect stage of data maturity?"
# docs = docsearch.similarity_search(query, include_metadata=True)

In [33]:
chain.run(input_documents=docs, question=query + " After your answer, include all of the lesson or sutra numbers from the entire context with the phrase: 'As found in the following sources: '. Say 'Sorry, I cannot find that answer, but you might try the following sources: ' if you do not have any source from the context provided. For both sources statement, do not include the 'Bhashya' title.")

' Hinduism prohibits suicide because it creates unseemly karma to be faced in the future. However, in cases of terminal illness, religious self-willed death through fasting, prayopavesha, is sometimes permitted. As found in the following sources: Lesson 229 from Living with Siva, Sutra 57 of the Nandinatha Sutras, and Sloka 35 from Dancing with Siva.'